# 

In [22]:
# !conda install -c conda-forge BeautifulSoup4 --yes
print('done')

Note: you may need to restart the kernel to use updated packages.
done


In [59]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Scrapping wiki

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')

### Converting the html table in a df

In [60]:
table = soup.find_all('table')[0]
column_names = ["Postal Code", "Borough", "Neighborhood"]
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df=(pd.DataFrame(l, columns=column_names))
df=df.replace('\n','',regex=True)
df=df[df.Borough != 'Not assigned'].reset_index(drop=True)
df=df.drop(df.index[0])

In [62]:
df.shape

(103, 3)

In [64]:
df.head()

,Postal Code,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
